In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import time
from collections import defaultdict
from tqdm import tqdm
from collections import Counter
from itertools import combinations

In [2]:
path = os.path.dirname(os.getcwd())
data_path = os.path.join(path, 'data', 'T10I4D100K.dat')
data_path

'd:\\Study\\KTH\\DM\\assignment\\Datamining_HW2\\data\\T10I4D100K.dat'

In [3]:
def read_dataset(
        file: str
) -> list[set[int]]:
    """
    This function reads from a file .dat assuming that on every row of the file there is a basket of items and returns
    the list of the baskets.

    :param file: the path to the input file
    :return: the list of baskets
    """

    with open(file, "r") as f:
        baskets: list[set[int]] = list(
            map(
                lambda basket: {int(item_id) for item_id in basket.split()},
                f.read().splitlines()
            )
        )
    f.close()

    return baskets

In [4]:
baskets = read_dataset(file=data_path)
print(baskets[0])

{448, 834, 164, 775, 328, 687, 240, 368, 274, 561, 52, 630, 825, 25, 538, 730}


In [5]:
for basket in baskets:
    print(basket)
    for item in basket:
        print(item)
        break
    break

{448, 834, 164, 775, 328, 687, 240, 368, 274, 561, 52, 630, 825, 25, 538, 730}
448


Singleton

In [6]:
def find_frequent_singletons(baskets, s=1, show=False):
    item_to_support = defaultdict(int)

    for basket in baskets:
        for item in basket:
            item_to_support[frozenset([item])] += 1

    if show:
        print(f'Data contains {len(item_to_support)} different items.')
        # support_values = list(item_to_support.values())
        # print(f'The average support is {sum(support_values) / len(support_values):.2f}')

    return {k: v for k, v in item_to_support.items() if v > s}


In [7]:
s = 1000


frequent_item_sets = find_frequent_singletons( baskets=baskets, s = s, show = True)


print(f'The most frequent singletons have been calculated. {len(frequent_item_sets)} singletons was/were found.')

Data contains 870 different items.
The most frequent singletons have been calculated. 375 singletons was/were found.


In [8]:
import itertools
def generate_candidates(precedent_item_sets,item_set_length,baskets):
    L1=set()
    ret=set()
    for item in precedent_item_sets:
        for val in item:
            L1.add(val)
    for basket in baskets:
        intersection=list(L1 & basket)
        candidates=list(itertools.combinations(intersection,item_set_length))
        for cand in candidates:
            prev_cand=itertools.combinations(list(cand),item_set_length-1)
            flag=True
            for prev_c in prev_cand:
                if frozenset(prev_c) not in precedent_item_sets:
                    flag=False
                    break
            if not flag:
                continue
            ret.add(frozenset(cand))
    return ret
            

    

In [9]:
def generate_candidate(precedent_item_sets, item_set_length):
    return {
        item1 | item2
        for item1 in precedent_item_sets
        for item2 in precedent_item_sets
        if len(item1 | item2) == item_set_length
    }


In [10]:
def filter(baskets, candidate_item_sets, item_set_length, s=1, show = False):
    item_set_to_support = Counter(
        frozenset(item_set)
        for basket in baskets
        for item_set in combinations(basket, item_set_length)
        if frozenset(item_set) in candidate_item_sets
    )
    # print("item_set_to_support", list(item_set_to_support)[0])
    if show:
        print("length", item_set_length)
        # print(f'The market contains {len(item_set_to_support)} different items.')


    return {k: v for k, v in item_set_to_support.items() if v > s}


In [11]:
precedent_frequent_item_sets = frequent_item_sets.keys()
print(precedent_frequent_item_sets)
item_set_length = 2
print("length", item_set_length)

dict_keys([frozenset({448}), frozenset({834}), frozenset({775}), frozenset({687}), frozenset({240}), frozenset({368}), frozenset({274}), frozenset({561}), frozenset({52}), frozenset({630}), frozenset({825}), frozenset({25}), frozenset({538}), frozenset({704}), frozenset({581}), frozenset({39}), frozenset({205}), frozenset({814}), frozenset({401}), frozenset({120}), frozenset({674}), frozenset({35}), frozenset({854}), frozenset({950}), frozenset({733}), frozenset({449}), frozenset({964}), frozenset({422}), frozenset({937}), frozenset({857}), frozenset({895}), frozenset({738}), frozenset({708}), frozenset({229}), frozenset({294}), frozenset({966}), frozenset({978}), frozenset({883}), frozenset({853}), frozenset({283}), frozenset({381}), frozenset({766}), frozenset({104}), frozenset({620}), frozenset({143}), frozenset({569}), frozenset({798}), frozenset({809}), frozenset({682}), frozenset({970}), frozenset({782}), frozenset({529}), frozenset({658}), frozenset({947}), frozenset({214}), fro

In [12]:
candidate_item_sets = generate_candidates(
            precedent_item_sets=precedent_frequent_item_sets,
            item_set_length=item_set_length,
            baskets=baskets
        )
print("{} candidates generated!".format(len(candidate_item_sets)))
print(list(candidate_item_sets)[0])

70125 candidates generated!
frozenset({413, 494})


In [13]:
new_frequent_item_sets = filter(
                baskets=baskets,
                candidate_item_sets=candidate_item_sets,
                item_set_length=item_set_length,
                s=s,
                show=True
            )
print(f'The most frequent {item_set_length}-item sets have been calculated. {len(new_frequent_item_sets)} {item_set_length}-item sets was/were found.')

length 2
The most frequent 2-item sets have been calculated. 9 2-item sets was/were found.


In [14]:
precedent_frequent_item_sets = new_frequent_item_sets.keys()
print(precedent_frequent_item_sets)
item_set_length = 3
print("length", item_set_length)

dict_keys([frozenset({704, 39}), frozenset({704, 825}), frozenset({825, 39}), frozenset({227, 390}), frozenset({789, 829}), frozenset({368, 829}), frozenset({217, 346}), frozenset({368, 682}), frozenset({722, 390})])
length 3


In [15]:
candidate_item_sets = generate_candidates(
            precedent_item_sets=precedent_frequent_item_sets,
            item_set_length=item_set_length,
            baskets=baskets
        )
print("{} candidates generated!".format(len(candidate_item_sets)))
print(list(candidate_item_sets)[0])

1 candidates generated!
frozenset({704, 825, 39})


In [16]:
new_frequent_item_sets = filter(
                baskets=baskets,
                candidate_item_sets=candidate_item_sets,
                item_set_length=item_set_length,
                s=s,
                show=True
            )
print(f'The most frequent {item_set_length}-item sets have been calculated. {len(new_frequent_item_sets)} {item_set_length}-item sets was/were found.')
print(list(new_frequent_item_sets))

length 3
The most frequent 3-item sets have been calculated. 1 3-item sets was/were found.
[frozenset({704, 825, 39})]


In [188]:
C1_raw=dict()
support_thre=len(baskets)*0.01
print("support_thre=", support_thre)

for basket in baskets:
    for item in basket:
        if item in C1_raw.keys():
            C1_raw[item]+=1
        else:
            C1_raw[item]=1
print(len(C1_raw))

C1=set()
for k,v in C1_raw.items():
    if v>=support_thre:
        C1.add(k)
del C1_raw
print(len(C1))

support_thre= 1000.0
870
375


In [198]:
C2_raw=dict()
for item1 in C1:
    for item2 in C1:
        if(item1==item2):
            break
        C2_raw[tuple({item1} | {item2})]=0
print(len(C2_raw))
# for basket in tqdm(baskets):
#     print("basket=",basket)
#     for k in C2_raw.keys():
#         print("k=",k)
#         if set(k) in basket:
#             print("k=",set(k))
#             C2_raw[k]+=1
#             break
    

# C2=set()
# for k,v in C2_raw.items():
#     if v>=support_thre:
#         C2.add(set(k))
# del C2_raw
# print(len(C2))
# print(C2)

70125


In [190]:
# C3_raw=dict()
# for item1 in C1:
#     for item2 in C2:
#         flag=True
#         for t in item2:
#             if {t}|item1 not in C2:
#                 flag=False
#                 break
#         if not flag:
#             continue
#         C3_raw[item1|item2]=0
# print(len(C3_raw))
# for basket in baskets:
#     for k in C3_raw.keys():
#         if k in basket:
#             C3_raw[k]+=1
# C3=set()
# for k,v in C3_raw.items():
#     if v>=support_thre:
#         C3.add(k)
# del C3_raw
# print(len(C3))